In [33]:
# Using kaleido package to generate static images
import numpy as np
from scipy.optimize import fsolve
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'


def system_of_equations_3d(vars, xr0):
    v, xp, xr = vars
    x = min(s * rho * xr + (1 - rho) * xp, 1)
    eta_x = 0.2 + 0.4 * x
    yp = 1 - xp
    yr = 1 - xr
    gr = 1 - (cr / (1 + np.exp(-1 * kr * (Tv - 5 * (1 - v) - Tc))))
    gp = 1 - (cp / (1 + np.exp(-1 * kp * (Tv - 5 * (1 - v) - Tc))))
    ir = ir0 * max(gr, 0)
    ip = ip0 * max(gp, 0)
    v_lag = v
    alpha_p = alpha_p0 + d / (1 + np.exp(-1 * omega * ((1 - xr) / (1 - xr0)) * (ir / ip) * (ip0 / ir0) - dc))
    fTf = fmax / (1 + np.exp(-1 * w * (-7.5 * v + 7.5 * v_lag - Tc)))
    ep_m = -1 * alpha_p + 0.5 * fTf + delta * (xp + (1 - h) * xr)
    ep_n = -0.5 * fTf + delta * (yp + (1 - h) * yr)
    er_m = -1 * alpha_r + 0.5 * fTf + delta * ((1 - h) * xp + xr)
    er_n = -0.5 * fTf + delta * ((1 - h) * yp + yr)

    dv_dt = 2 * (1 - 0.01 * (Tv - 23 - 5 * v) ** 2) * eta_x * v * (1 - v) - 0.2 * v
    dxp_dt = kappa * xp * (1 - xp) * (ep_m - ep_n) + (1 - h) * kappa * (xr * max(er_m - ep_n,0) * (1 - xp) - (1 - xr) * max(er_n - ep_m,0) * xp)
    #print(dxp_dt)
    dxr_dt = kappa * xr * (1 - xr) * (er_m - er_n) + (1 - h) * kappa * (xp * max(ep_m - er_n,0) * (1 - xr) - (1 - xp) * max(ep_n - er_m,0) * xr)

    return [dv_dt, dxp_dt, dxr_dt]

def v_solutions(v, xp, xr):
    x = s * rho * xr + (1-rho) * xp
    eta_x = 0.2+0.4 * x
    return (2 * (1 - 0.01 * (Tv - 23 - 5 * v) ** 2) * eta_x * v * (1 - v)) - 0.2 * v

def xp_solutions(v,xp,xr, xr0):
    yp = 1 - xp
    yr = 1 - xr
    gr = 1 - (cr / (1 + np.exp(-1 * kr * (Tv - 5 * (1 - v) - Tc))))
    # print("gr", gr)
    gp = 1 - (cp / (1 + np.exp(-1 * kp * (Tv - 5 * (1 - v) - Tc))))
    # print("gp", gp)
    ir = ir0 * max(gr, 0)
    ip = ip0 * max(gp, 0)
    v_lag = v
    alpha_p = alpha_p0 + d / (1 + np.exp(-1 * omega * ((1 - xr) / (1 - xr0)) * (ir / ip) * (ip0 / ir0) - dc))
    fTf = fmax / (1 + np.exp(-1 * w * (-7.5 * v + 7.5 * v_lag - Tc)))
    ep_m = -1 * alpha_p + 0.5 * fTf + delta * (xp + (1 - h) * xr)
    ep_n = -0.5 * fTf + delta * (yp + (1 - h) * yr)
    er_m = -1 * alpha_r + 0.5 * fTf + delta * ((1 - h) * xp + xr)
    er_n = -0.5 * fTf + delta * ((1 - h) * yp + yr)
    return kappa * xp * (1-xp) * (ep_m - ep_n) + (1 - h) * kappa * (xr * max(er_m - ep_n,0) * (1 - xp) - (1 - xr) * max(er_n - ep_m,0) * xp)

def xr_solutions(v, xp, xr, xr0):
    yp = 1 - xp
    yr = 1 - xr
    gr = 1 - (cr / (1 + np.exp(-1 * kr * (Tv - 5 * (1 - v) - Tc))))
    gp = 1 - (cp / (1 + np.exp(-1 * kp * (Tv - 5 * (1 - v) - Tc))))
    ir = ir0 * max(gr, 0)
    ip = ip0 * max(gp, 0)
    v_lag = v
    alpha_p = alpha_p0 + d / (1 + np.exp(-1 * omega * ((1 - xr) / (1 - xr0)) * (ir / ip) * (ip0 / ir0) - dc))
    fTf = fmax / (1 + np.exp(-1 * w * (-7.5 * v + 7.5 * v_lag - Tc)))
    ep_m = -1 * alpha_p + 0.5 * fTf + delta * (xp + (1 - h) * xr)
    ep_n = -0.5 * fTf + delta * (yp + (1 - h) * yr)
    er_m = -1 * alpha_r + 0.5 * fTf + delta * ((1 - h) * xp + xr)
    er_n = -0.5 * fTf + delta * ((1 - h) * yp + yr)
    return kappa * xr * (1 - xr) * (er_m - er_n) + (1 - h) * kappa * (xp * max(ep_m - er_n,0) * (1 - xr) - (1 - xp) * max(ep_n - er_m,0) * xr)

def find_xp(v, xr, xr0):
    initial_guesses = [0.3,0.7]
    xp_solutions = []
    for guess in initial_guesses:
        xr0=guess
        #xp_solution = fsolve(lambda xp: xp_solutions(v, xp, xr, xr0), initial_guesses)
        root = fsolve(lambda xp: xp_solutions(v, xp, xr, xr0), guess)[0]
        #print("Xp", xp_solution)
        xp_solutions.append(root)
    return xp_solutions

def find_v(xp, xr):
    initial_guesses = [0.3,0.7]
    v_solution = fsolve(lambda v: v_solutions(v, xp, xr), initial_guesses)
    return v_solution

def find_xr(v, xp, xr0):
    initial_guesses = [0.3,0.7]
    xr_solutions=[]
    for guess in initial_guesses:
        xr0=guess
        #xr_solution = fsolve(lambda xr: xr_solutions(v,xp,xr, xr0), initial_guesses)
        root = fsolve(lambda xr: xr_solutions(v,xp,xr,xr0), guess)[0]
        #print("Xr", xr_solution)
        xr_solutions.append(root)
    return xr_solutions

for val in [0,0.5,1]:
    s = 1
    rho = 0.25
    kappa = 0.05
    Tv = 30
    Tc = 0.5
    kr = 1
    kp = 1.5
    cr = 0.4
    cp = 0.85
    omega = 3
    w = 3
    fmax = 5
    d = 5
    dc = 1.5
    alpha_p0 = 1
    ir0 = 5
    ip0 = 3.5
    delta = 1
    alpha_r = 0.5
    h=val
    
    v_vals = np.linspace(-0.4, 1.4, 60)
    xr_vals = np.linspace(-0.4, 1.4, 60)
    xp_vals = np.linspace(-0.4,1.4,60)
    V, XP, XR = np.meshgrid(v_vals, xp_vals, xr_vals, indexing='ij')
            
    v_solutions = []
    xp_solutions = []
    xr_solutions = []

    guesses3d = [[0,0,0], [0.1,0.1,0.1], [0.2,0.2,0.2], [0.3,0.3,0.3], [0.4,0.4,0.4], [0.5,0.5,0.5], [0.6,0.6,0.6], [0.7,0.7,0.7], [0.8,0.8,0.8], [0.9,0.9,0.9]]
    for guess in guesses3d:
        xr0 = guess[0]
        solution = fsolve(system_of_equations_3d, guess, args=(xr0))
        #print(solution)
        if solution[0]>=-0.1 and solution[0]<=1.1 and solution[1]>=-0.1 and solution[1]<=1.1 and solution[2]>=-0.1 and solution[2]<=1.1:
            v_solutions.append(solution[0])
            xp_solutions.append(solution[1])
            xr_solutions.append(solution[2])
    
    v_sparse_vals = np.linspace(0.1,0.9,3)
    xp_sparse_vals = np.linspace(0.1,0.9,3)
    xr_sparse_vals = np.linspace(0.1,0.9,3)
    
    V_sparse, XP_sparse, XR_sparse = np.meshgrid(v_sparse_vals, xp_sparse_vals, xr_sparse_vals, indexing='ij')
    
    dv_dt_sparse = np.zeros(V_sparse.shape)
    dxp_dt_sparse = np.zeros(XP_sparse.shape)
    dxr_dt_sparse = np.zeros(XR_sparse.shape)
    
    for i in range(V_sparse.shape[0]):
        for j in range(V_sparse.shape[1]):
            for k in range(V_sparse.shape[2]):
                v = V_sparse[i, j, k]
                xp = XP_sparse[i, j, k]
                xr = XR_sparse[i, j, k]
                dv, dxp, dxr = system_of_equations_3d([v, xp, xr], 0.3)
                dv_dt_sparse[i, j, k] = dv
                dxp_dt_sparse[i, j, k] = dxp
                dxr_dt_sparse[i, j, k] = dxr
    
    
    dv_dt = np.zeros(V.shape)
    dxp_dt = np.zeros(XP.shape)
    dxr_dt = np.zeros(XR.shape)
    for i in range(V.shape[0]):
        for j in range(V.shape[1]):
            for k in range(V.shape[2]):
                v = V[i,j,k]
                xp=XP[i,j,k]
                xr =XR[i,j,k]
                dv, dxp, dxr = system_of_equations_3d([v, xp, xr], 0.3)
                dv_dt[i, j, k] = dv
                dxp_dt[i, j, k] = dxp
                #print(dxp_dt)
                dxr_dt[i, j, k] = dxr
    
    fig = go.Figure()
    fig.add_trace(go.Isosurface(
        x=V.flatten(),
        y=XP.flatten(),
        z=XR.flatten(),
        value=dv_dt.flatten(),
        isomin=-0.001,
        isomax=0.001,
        surface_count=1,
        caps=dict(x_show=False, y_show=False, z_show=False),
        colorscale=["#4C72B0","#4C72B0"],
        opacity=0.4,
        showscale=False,
    ))
    
    
    fig.add_trace(go.Isosurface(
        x=V.flatten(),
        y=XP.flatten(),
        z=XR.flatten(),
        value=dxr_dt.flatten(),
        isomin=-0.001,
        isomax=0.001,
        surface_count=1,
        caps=dict(x_show=False, y_show=False, z_show=False),
        colorscale=["#DD8452","#DD8452"],
        opacity=0.4,
        showscale=False,
    ))
    
    fig.add_trace(go.Isosurface(
        x=V.flatten(),
        y=XP.flatten(),
        z=XR.flatten(),
        value=dxp_dt.flatten(),
        isomin=-0.001,
        isomax=0.001,
        surface_count=1,
        caps=dict(x_show=False, y_show=False, z_show=False),
        colorscale=["#55A868","#55A868"],
        opacity=0.4,
        showscale=False,
    ))
    
    fig.add_trace(go.Scatter3d(
        x=v_solutions,
        y=xp_solutions,
        z=xr_solutions,
        mode='markers',
        marker=dict(
            size=15,
            color='black',
            opacity=0.8
        )
    ))
    
    
    u = dv_dt_sparse.flatten()
    v = dxp_dt_sparse.flatten()
    w = dxr_dt_sparse.flatten()
    
    # Compute magnitudes
    magnitude = np.sqrt(u**2 + v**2 + w**2)
    
    magnitude_safe = np.where(magnitude == 0, 0.05, magnitude)
    
    # Normalize components
    u_norm = u / magnitude_safe
    v_norm = v / magnitude_safe
    w_norm = w / magnitude_safe
    
    angles = np.arctan2(v_norm, u_norm)
    
    angles_normalized = (angles + np.pi) / (2 * np.pi)
    
    fig.add_trace(go.Cone(
        x=V_sparse.flatten(),
        y=XP_sparse.flatten(),
        z=XR_sparse.flatten(),
        u=u_norm,
        v=v_norm,
        w=w_norm,
        colorscale=['white','white'],
        sizemode="absolute",
        sizeref=0.2,
        showscale=False,
    ))
    
    combined = np.sqrt(dv_dt**2 + dxp_dt**2 + dxr_dt**2)

    fig.update_layout(
    scene=dict(
        xaxis=dict(
            title=dict(text="v", font=dict(size=30)),
            tickfont=dict(size=16),
            range=[0, 1],
        ),
        yaxis=dict(
            title=dict(text="x<sub>P</sub>", font=dict(size=30)),
            tickfont=dict(size=16),
            range=[0, 1],
        ),
        zaxis=dict(
            title=dict(text="x<sub>R</sub>", font=dict(size=30)),
            tickfont=dict(size=16),
            range=[0, 1],
        ),
    ),
    paper_bgcolor="white",
    plot_bgcolor="white",
    #margin=dict(l=400, r=400, t=200, b=800),
    #margin=dict(l=100,r=100,t=100,b=100),
)
    fig.update_xaxes(automargin=True)
    fig.update_yaxes(automargin=True)
    #fig.update_zaxes(automargin=True)
    
    #fig.show()
    fig.write_html(f"interactive_plotly_ploth{h}.html")
    fig.write_image(f"static_plotly_ploth{h}.png", width=2000, height=2000, scale=3)


/tmp/ipykernel_3847744/1584761407.py:134: RuntimeWarning:

The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.

